In [1]:
from elasticsearch import Elasticsearch
import json
from tqdm.auto import tqdm
from openai import OpenAI

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q1. Running Elastic

In [17]:
es_client = Elasticsearch('http://localhost:9200')
print(f"the version.build_hash value is '{es_client.info()['version']['build_hash']}'")

the version.build_hash value is '42f05b9372a9a4a470db3b52817899b99a76ee73'


In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "homework-questions"

es_client.indices.create(index=index_name, body=index_settings)

## Q2. Indexing the data

In [5]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3. Searching

In [6]:
question = "How do I execute a command in a running docker container?"

In [7]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                        "query": question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

In [8]:
print(f"the score for the top ranking result is {response['hits']['hits'][0]['_score']}")

the score for the top ranking result is 84.050095


## Q4. Filtering

In [34]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

In [35]:
print(f"the 3rd question returned by elastic search is '{response['hits']['hits'][2]['_source']['question']}'")

the 3rd question returned by elastic search is 'How do I copy files from a different folder into docker container’s working directory?'


## Q5. Building a prompt

In [43]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    context_template = """
Q: {question}
A: {text}
""".strip()
    
    for doc in search_results:
        context += context_template.format(question=doc['question'], text=doc['text']) + '\n\n'
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [44]:
prompt = build_prompt(query=question, search_results=[hit['_source'] for hit in response['hits']['hits']])

In [45]:
len(prompt)

1462

## Q6. Tokens

In [46]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 748.1 kB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.9 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 16.1 MB/s eta 0:00:00m eta 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [48]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [52]:
print(f"The prompt has {len(encoding.encode(prompt))} tokens")

The prompt has 322 tokens
